In [1]:
import numpy as np
import pandas as pd
import gdal
from matplotlib import pyplot as plt
import os
import re
import rasterio
from rasterio.merge import merge
from rasterio.plot import show

# Declare direcory with all the 10 days tiles tif file

In [2]:
directory='/home/gbessardon/DATA/laialebdomodis/tilesDIR'

# Declare output dir

In [3]:
outputdir='/home/gbessardon/DATA/laialebdomodis/mosaicDIR'

# Identify files visible nir files and days

## Identify days in the days in a filename

In [4]:
def finddaysinfilename(fn,pattern):
    string=re.search(pattern,fn).group()
    emp_str=""
    for m in string:
        if m.isdigit():
            emp_str=emp_str+m
    return emp_str

## Create a dataframe that groups the filename in 10-days period

In [5]:
def Create_dataframe(directory,filestart,filend='tif',dayspattern='jd\d\d\d\d'):
    files=os.listdir(directory)
    vis_fn=[os.path.join(directory,f) for f in files if (f.startswith(filestart) and f.endswith(filend))]
    visdayslist= [finddaysinfilename(f,dayspattern) for f in vis_fn]
    data = {'filename':vis_fn,
        'julian10days':visdayslist}
    df=pd.DataFrame(data)
    grouped=df.groupby(['julian10days'])
    return df,grouped

In [6]:
def writemosaic(input_list,out_fp):
    src_files_to_mosaic = []
    for f in list(input_list):
        src=rasterio.open(f)
        src_files_to_mosaic.append(src)
    mosaic, out_trans = merge(src_files_to_mosaic,nodata=int(src.nodata))
    out_meta = src.meta.copy()

    # Update the metadata
    out_meta.update({"driver": out_meta.get('driver'),
                  "height": mosaic.shape[1],
                  "width": mosaic.shape[2],
                  "transform": out_trans,
                  "nodata":int(src.nodata),
                 }
                )
    with rasterio.open(out_fp, "w", **out_meta) as dest:
        dest.write(mosaic)
    return (int(src.nodata),src.read_crs())

In [7]:
def reproject_resample_mosaic(inputfn,outputfn,MODISsrs,nodata,
                              ECOSGres=0.00277777777777,ECOSGnodata=0,ECOSGepsg='EPSG:4326'):
    ds=gdal.Open(inputfn)
    opt=gdal.WarpOptions(srcSRS=MODISsrs,dstSRS=ECOSGepsg,srcNodata=nodata,dstNodata=ECOSGnodata,xRes=ECOSGres,yRes=-ECOSGres)
    gdal.Warp(outputfn,ds,options=opt)
    return

In [8]:
def Loopover(grouped,outdir,filestart):
    for name,group in grouped:
        output_fn=filestart+str(name)+'.tif'
        input_list=list(group.filename.values)
        nodata,transfo=writemosaic(input_list,os.path.join(outdir,output_fn))
        ECOSGproj_fn=output_fn.replace('.tif','ecosgproj.tif')
        reproject_resample_mosaic(os.path.join(outdir,output_fn),os.path.join(outdir,ECOSGproj_fn),transfo,nodata)
        print(ECOSGproj_fn)
    return(len(os.listdir(outdir)))

# Main

In [9]:
visdf,vis_gr=Create_dataframe(directory,'vis_h',filend='tif',dayspattern='jd\d\d\d\d')
vis_sf_df,vis_sf_gr=Create_dataframe(directory,'vis_sf_h',filend='tif',dayspattern='jd\d\d\d\d')
nirdf,nir_gr=Create_dataframe(directory,'nir_h',filend='tif',dayspattern='jd\d\d\d\d')
nir_sf_df,nir_sf_gr=Create_dataframe(directory,'nir_sf_h',filend='tif',dayspattern='jd\d\d\d\d')

In [10]:
if not os.path.isdir(outputdir):
    os.mkdir(outputdir)

In [11]:
Loopover(vis_gr,outputdir,'mosaic_VIS')
Loopover(vis_sf_gr,outputdir,'mosaic_VIS_SF')
Loopover(nir_gr,outputdir,'mosaic_NIR')
Loopover(nir_sf_gr,outputdir,'mosaic_NIR_SF')

mosaic_VIS0625ecosgproj.tif
mosaic_VIS0705ecosgproj.tif
mosaic_VIS0715ecosgproj.tif
mosaic_VIS0725ecosgproj.tif
mosaic_VIS0805ecosgproj.tif
mosaic_VIS0815ecosgproj.tif
mosaic_VIS0825ecosgproj.tif
mosaic_VIS0905ecosgproj.tif
mosaic_VIS_SF0625ecosgproj.tif
mosaic_VIS_SF0705ecosgproj.tif
mosaic_VIS_SF0715ecosgproj.tif
mosaic_VIS_SF0725ecosgproj.tif
mosaic_VIS_SF0805ecosgproj.tif
mosaic_VIS_SF0815ecosgproj.tif
mosaic_VIS_SF0825ecosgproj.tif
mosaic_VIS_SF0905ecosgproj.tif
mosaic_NIR0625ecosgproj.tif
mosaic_NIR0705ecosgproj.tif
mosaic_NIR0715ecosgproj.tif
mosaic_NIR0725ecosgproj.tif
mosaic_NIR0805ecosgproj.tif
mosaic_NIR0815ecosgproj.tif
mosaic_NIR0825ecosgproj.tif
mosaic_NIR0905ecosgproj.tif


48

In [13]:
nir_sf_df

,filename,julian10days
